In [1]:
import os
import toml
from dotenv import load_dotenv
from src.document_loader import DocumentLoader
from src.embeddings import Embeddings
from src.vector_store import VectorStore
from src.retriever import Retriever
from src.llm import LLM
from src.rag_agent import RAGAgent

[nltk_data] Downloading package punkt to /home/chollar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
load_dotenv()

env = os.getenv('ENV', 'development')

# Load configuration
with open(f'config/config.{env}.toml', 'r') as file:
    config = toml.load(file)
    
model_id = config['model']['id']
document_directory = config['documents']['directory']
embedding_directory = os.path.join(document_directory, "embeddings")
    
loader = DocumentLoader(document_directory)
titles, documents = loader.load_documents()
    
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not found in environment variables")

embeddings = Embeddings(model_id=model_id, HUGGINGFACE_API_KEY=huggingface_api_key)
document_embeddings, chunked_texts_with_titles = embeddings.get_embeddings(titles, documents, embedding_directory=embedding_directory)

In [3]:
embedding_dimension = len(document_embeddings[0])
vector_store = VectorStore(dimension=embedding_dimension)
vector_store.add_documents(chunked_texts_with_titles, document_embeddings)

retriever = Retriever(vector_store, embeddings)
llm = LLM(config['llm']['api_url'])

rag_agent = RAGAgent(retriever=retriever, llm=llm)

In [4]:
query = "When he felt a hand on his shoulder, who's hand was it and what did he say?"
query = "I have a question from Dracula. After Mrs. Harker came into the room, why did the two men sitting on the bed move?"
prompt = rag_agent.get_prompt(query=query)
prompt

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


'Context: Dracula_Chunk_1862: To work! To work! When Dr. Van Helsing and Dr. Seward had come back from seeing poor\nRenfield, we went gravely into what was to be done. Dracula_Chunk_2188: Something is shifting\nfrom me in some strange way, and I feel freer than I have been of late!”\nThe three younger men looked happier at the moment as they seemed to\nrealise the significance of her words; but Van Helsing and I, turning to\neach other, met each a grave and troubled glance. We said nothing at the\ntime, however. When the three men had gone out to their tasks Van Helsing asked Mrs.\nHarker to look up the copy of the diaries and find him the part of\nHarker’s journal at the Castle. Dracula_Chunk_2290: Of course he wanted to be with me; but then the\nboat service would, most likely, be the one which would destroy the ...\nthe ... the ... Vampire. (Why did I hesitate to write the word?) He was\nsilent awhile, and during his silence Dr. Van Helsing spoke:--\n\n“Friend Jonathan, this is to y

In [5]:
answer = rag_agent.answer(query=query)
answer

'The two men sitting on the bed moved after Mrs. Harker entered the room because they were looking at her and were concerned about her well-being.</s>'